# Network Analysis

In [ ]:
# https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python

In [414]:
import pandas as pd
from collections import defaultdict
import itertools

import networkx as nx
import nxviz
import matplotlib.pyplot as plt

from nxviz.plots import MatrixPlot, ArcPlot, CircosPlot
from community import community_louvain

from pyvis.network import Network
#import plotly.offline as py
#import plotly.graph_objects as go

In [740]:
def create_edgelist(entity_list, retweets):
    """
    Creates a weighted edgelist based on entity co-occurence.
    
        Parameters:
            entity_list (list): List of lists of entities occuring together.
        
        Returns:
            df_edgelist (dataframe): Pandas dataframe that shows number of co-occurances of different entities. 
    """
    
    print("\nCreating Edgelist...\n\n")
    
    cooccurence_dict = defaultdict(lambda: defaultdict(int))
    
    # Iterate over all entity co-occurences
    for cooccurence, retweet in zip(entity_list,retweets):
    
    # Check whether more than one entity is available
        if len(cooccurence) > 1:
    
        # Get all possible combinations of entities
            combinations = list(itertools.combinations(sorted(cooccurence), 2))
            
    
            # Increment count of entity co-occurences by retweet count + 1
            for combination in combinations:
                cooccurence_dict[combination]["Co_Occurence"] += 1
                cooccurence_dict[combination]["Retweet"] += (retweet+1)             
        
    # Convert dict to dataframe
    #df_edgelist = pd.Series(cooccurance_dict).reset_index()
    #df_edgelist.columns = ['Entity_1', 'Entity_2', 'Weight']
    
    df_edgelist = pd.DataFrame.from_dict(cooccurence_dict, orient="index")
    df_edgelist.reset_index(inplace=True)
    df_edgelist.rename({"level_0":"Entity_1","level_1":"Entity_2"},axis=1, inplace=True)
    df_edgelist = df_edgelist[df_edgelist["Entity_1"] != df_edgelist["Entity_2"]]
    
    # Return edgelist dataframe
    return df_edgelist

In [416]:
def get_entity_count(series):
    dict_count = defaultdict(int)
    
    for entities in series:
        for entity in entities:
            if len(entity) > 0:
                dict_count[entity] += 1
            
    return dict_count

In [ ]:
def create_overall_graph(df,df_edgelist,entity = "hashtags"):
    print("\nCreating the Overall Graph...\n\n")

    G = nx.Graph()
    
    for entity_1,entity_2,weight in zip(df_edgelist["Entity_1"],
                                        df_edgelist["Entity_2"],
                                        df_edgelist["Weight"]):
        G.add_edge(entity_1,entity_2, weight = weight, width=weight)
    
    
    print(nx.info(G_hashtag))
    print("\n----------------------------------------------\n\n")

    entity_count = get_entity_count(df[entity])
    degree_centrality = nx.degree_centrality(G)
    eigenvector_centrality = nx.eigenvector_centrality(G, weight = "weight")
    size = {k:v/max(degree_centrality.values())*30 for (k,v) in degree_centrality.items()}

    nx.set_node_attributes(G, entity_count, "entity_count" )
    nx.set_node_attributes(G, degree_centrality, "Degree Centrality")
    nx.set_node_attributes(G, eigenvector_centrality, "Eigenvector Centrality")
    nx.set_node_attributes(G, size, "size")
    
    df_node_attributes = pd.DataFrame.from_dict(dict(G.nodes(data=True)),
                                            orient="index").reset_index().rename({"index":"entity"}, axis = 1)
    df_node_attributes = df_node_attributes.sort_values(by="Degree Centrality", ascending=False).reset_index(drop=True)
    
    print(f"Most important {entity} in the network:")
    print(df_node_attributes[["entity","entity_count","Degree Centrality"]].head(15))
    print("\n----------------------------------------------\n\n")
    
    return G, df_node_attributes

In [762]:
def create_subgraph(df_node_attributes, G, node_count = 50, filename = "abc.html"):
    print("\nCreating Subgraph...\n\n")
    
    cut_off = df_node_attributes.loc[node_count,"Degree Centrality"]
    selected_nodes = [n for n,v in G.nodes(data=True) if v["Degree Centrality"]>cut_off]
    
    G_sub = G.subgraph(selected_nodes)
    
    partition = community_louvain.best_partition(G_sub, random_state=42)
    print(f"Identified {len(set(partition.values()))} clusters.")
    
    colors_partition = {k:v for (k,v) in colors.items() if k < len(set(partition.values()))}
    
    node_colors = {k:colors_partition[v] for (k,v) in partition.items()}
    
    nx.set_node_attributes(G_sub, partition, "Partiton")
    nx.set_node_attributes(G_sub, node_colors, "color")
    
    print("\nPlotting Graph...\n\n")
    n = Network(height = "1500px", width ="100%", font_color ="#616a6b ")
    #n.show_buttons()
    n.from_nx(G_sub)
    n.barnes_hut()
    n.show_buttons(filter_=["nodes","edges","physics"])
    n.show(filename)
    print("\nGraph succesfully plotted\n\n")

In [763]:
def all_in_one(df, entity, filename,node_count=50, weighting = "Retweet"):
    
    df_edgelist = create_edgelist(list(df[entity]), list(df["retweet_count_sum"]))
    df_edgelist["Weight"] = df_edgelist[weighting]/(max(df_edgelist[weighting])/25)
    df_edgelist = df_edgelist.sort_values(by="Weight", ascending=False)
    
    print(f"Most frequently co-occuring hashtags sorted by {weighting}:")
    print(df_edgelist[["Entity_1","Entity_2","Co_Occurence","Retweet"]].head(15))
    print("\n----------------------------------------------\n\n")
    
    G, df_node_attributes = create_overall_graph(df,
                                                 df_edgelist,
                                                 entity = entity)
    
    create_subgraph(df_node_attributes, G, node_count = node_count, filename = filename)

In [417]:
colors = {0:"#6495ED",
          1:"#9FE2BF",
          2:"#FFBF00",
          3:"#717d7e",
          4:"#cb4335",
          5:"#7d3c98",
          6:"#273746",
          7:"#196f3d "}

## Import the data

In [418]:
df_greece = pd.read_csv("df_greece_clean.csv",
                        converters = {"hashtags": lambda x: x.strip("[]").replace("'","").split(", "),
                                      "mentions": lambda x: x.strip("[]").replace("'","").split(", "),
                                      "annotations": lambda x: x.strip("[]").replace("'","").split(", ")})
# Drop unnecessary index column
df_greece.drop("Unnamed: 0", axis=1, inplace=True)
df_greece.head()

,source,text,lang,id,created_at,author_id,retweet_count,reply_count,like_count,quote_count,...,migrant,immigrant,asylum_seeker,other,date,text_coherent,retweet_count_sum,count,text_alphanum,text_stm
0,Hootsuite Inc.,PA Ambassador in Bosnia &amp; Herzegovina says...,en,1227019550912372737,2020-02-11 00:00:32+00:00,81136269,1,0,1,0,...,False,False,False,False,2020-02-11,PA Ambassador in Bosnia & Herzegovina says lif...,1,1,pa ambassador in bosnia herzegovina says life...,ambassador bosnia herzegovina say life europe ...
1,u.fooo.ooo,[🔴 NEWS] Greece plans floating sea border wall...,en,1227019556167864321,2020-02-11 00:00:33+00:00,1052191553802854407,0,0,0,0,...,False,False,False,False,2020-02-11,[ NEWS] Greece plans floating sea border wall ...,0,1,news greece plans floating sea border wall to...,news greece plan floating border wall keep
2,Twitter Web Client,Latest Battle for Idlib Could Send Another Wav...,en,1227021374780313601,2020-02-11 00:07:47+00:00,18570470,0,0,1,1,...,False,False,False,False,2020-02-11,Latest Battle for Idlib Could Send Another Wav...,8,5,latest battle for idlib could send another wav...,latest battle idlib could send another wave eu...
3,Tweepsmap,UNHCR calls for decisive action to end alarmin...,en,1227021789525614594,2020-02-11 00:09:26+00:00,62632306,0,0,0,0,...,False,False,False,False,2020-02-11,UNHCR calls for decisive action to end alarmin...,9,3,unhcr calls for decisive action to end alarmin...,unhcr call decisive action alarming condition ...
4,Twitter for iPhone,"It is not your own feet sinking in the mud, bo...",en,1227022233484308481,2020-02-11 00:11:12+00:00,2729959018,9,1,29,1,...,False,False,False,False,2020-02-11,"It is not your own feet sinking in the mud, boy.",9,1,it is not your own feet sinking in the mud boy.,foot sinking


In [750]:
df_tigray = pd.read_csv("df_tigray_clean.csv",
                        converters = {"hashtags": lambda x: x.strip("[]").replace("'","").split(", "),
                                      "mentions": lambda x: x.strip("[]").replace("'","").split(", "),
                                      "annotations": lambda x: x.strip("[]").replace("'","").split(", ")})
# Drop unnecessary index column
df_tigray.drop("Unnamed: 0", axis=1, inplace=True)
df_tigray.head()

,source,text,lang,id,created_at,author_id,retweet_count,reply_count,like_count,quote_count,...,migrant,immigrant,asylum_seeker,other,date,text_coherent,retweet_count_sum,count,text_alphanum,text_stm
0,Twitter Web App,@FilippoGrandi Thank you for your statement......,en,1349868943800897537,2021-01-15 00:00:10+00:00,1325094884680765445,1,0,1,0,...,False,False,False,False,2021-01-15,Thank you for your statement . . . Now what AC...,1,1,thank you for your statement . . . now what ac...,thank statement action taken civilian tigray u...
1,Twitter for iPhone,Action Now\n#HumanitarianCorridor Now\n#HumanR...,en,1349868980102696963,2021-01-15 00:00:18+00:00,1296598454,1,0,1,1,...,False,False,False,False,2021-01-15,Action Now humanitarian corridor Now,1,1,action now humanitarian corridor now,action humanitarian corridor
2,Twitter Web App,"""@AUC_MoussaFaki has disregarded and violated ...",en,1349869527773933569,2021-01-15 00:02:29+00:00,1331304034230292482,0,0,0,0,...,False,False,False,False,2021-01-15,""" Moussa Faki Mahamat has disregarded and viol...",73,55,moussa faki mahamat has disregarded and viola...,moussa faki mahamat disregarded violated inter...
3,Twitter for iPhone,@Refugees @zeaxumawit @FilippoGrandi But @anto...,en,1349870752070311936,2021-01-15 00:07:21+00:00,1325975568882479104,0,0,0,0,...,False,False,False,False,2021-01-15,But António Guterres said there are no Eritrea...,0,1,but antnio guterres said there are no eritrean...,guterres said eritrean troop ethiopia need ret...
4,Twitter Web App,We call for global solidarity to STOP killings...,en,1349871059835744256,2021-01-15 00:08:34+00:00,1327215334097608704,4,0,8,1,...,False,False,False,False,2021-01-15,We call for global solidarity to STOP killings...,4,1,we call for global solidarity to stop killings...,call global solidarity stop killing abduction ...


In [751]:
df_rohingya = pd.read_csv("df_rohingya_clean.csv",
                        converters = {"hashtags": lambda x: x.strip("[]").replace("'","").split(", "),
                                      "mentions": lambda x: x.strip("[]").replace("'","").split(", "),
                                      "annotations": lambda x: x.strip("[]").replace("'","").split(", ")})
# Drop unnecessary index column
df_rohingya.drop("Unnamed: 0", axis=1, inplace=True)
df_rohingya.head()

,source,text,lang,id,created_at,author_id,retweet_count,reply_count,like_count,quote_count,...,refugee,migrant,immigrant,asylum_seeker,other,text_coherent,retweet_count_sum,count,text_alphanum,text_stm
0,Twitter Web App,"For #Rohingya Survivors in Bangladesh, Artwork...",en,1373792416126402560,2021-03-22 00:23:30+00:00,77844813,2,1,2,0,...,True,False,False,False,False,"For rohingya Survivors in Bangladesh, Artwork ...",2,1,for rohingya survivors in bangladesh artwork b...,rohingya survivor bangladesh artwork bear witn...
1,dlvr.it,AstraZeneca dispels Indonesian Muslim concerns...,en,1373800977778700288,2021-03-22 00:57:31+00:00,1898083759,1,0,0,0,...,True,False,False,False,False,AstraZeneca dispels Indonesian Muslim concerns...,1,1,astrazeneca dispels indonesian muslim concerns...,astrazeneca dispels indonesian muslim concern ...
2,Twitter for Android,@prabha_j @MehHarshil @derekobrienmp I think u...,en,1373802051524730880,2021-03-22 01:01:47+00:00,1209116380257112064,0,0,1,0,...,False,True,False,False,False,I think u are one of the illegally migrant Roh...,0,1,i think u are one of the illegally migrant roh...,think illegally rohingya bangladesh better kee...
3,Twitter for Android,India seals Myanmar border amid strains over r...,en,1373802536579174401,2021-03-22 01:03:43+00:00,1032998054297780224,0,0,0,0,...,True,False,False,False,False,India seals Myanmar border amid strains over r...,0,1,india seals myanmar border amid strains over r...,india seal myanmar border amid strain crisis
4,Twitter for iPhone,"Fleeing coup, Myanmar police refugees in India...",en,1373804367757807619,2021-03-22 01:10:59+00:00,15552861,1,0,1,0,...,True,False,False,False,False,"Fleeing coup, Myanmar police refugees in India...",1,1,fleeing coup myanmar police refugees in india ...,fleeing coup myanmar police india seek asylum ...


In [752]:
all_in_one(df_greece,"hashtags","greece_hashtags.html", weighting="Co_Occurence")


Creating Edgelist...


Most frequently co-occuring hashtags sorted by Co_Occurence:
                Entity_1          Entity_2  Co_Occurence  Retweet
48                greece            turkey          3955    44678
575               greece          refugees          2645    19062
1373            refugees            turkey          2586    15474
102                syria            turkey          2345    16120
291                   eu            turkey          1820     9683
107             refugees             syria          1686     8280
218                   eu            greece          1543    12143
226                idlib             syria          1468    12428
37                europe            turkey          1401     9012
1496             erdogan            turkey          1312    15234
170               greece          migrants          1288     6837
155                   eu          refugees          1232     5203
202             migrants            turkey          1141 

In [753]:
all_in_one(df_greece,"mentions","greece_mentions_retweet.html",weighting="Co_Occurence")


Creating Edgelist...


Most frequently co-occuring hashtags sorted by Co_Occurence:
              Entity_1         Entity_2  Co_Occurence  Retweet
21961    eucopresident      vonderleyen           522     3471
1209     eu_commission      vonderleyen           470     1471
52728    eucopresident  primeministergr           341      726
125           refugees               un           321      863
21990     ep_president      vonderleyen           303      713
74193     ep_president    eucopresident           270      632
1180           skynews    stone_skynews           259      330
29609    eu_commission    eucopresident           249      632
1082   mindedmusically       the_leaver           210      227
969        jag11814459  mindedmusically           210      227
977        jag11814459       the_leaver           210      227
425         begentle50         dw2essex           209      226
872           dw2essex       the_leaver           209      226
437         begentle50  mindedmus

In [764]:
all_in_one(df_tigray,"hashtags","tigray_hashtags.html", weighting="Co_Occurence")


Creating Edgelist...


Most frequently co-occuring hashtags sorted by Co_Occurence:
                 Entity_1        Entity_2  Co_Occurence  Retweet
5                  tigray  tigraygenocide          2139    25528
41               ethiopia          tigray          1201    16951
393       bidentakeaction  tigraygenocide          1175    10580
6876  allowaccesstotigray  tigraygenocide          1131    15116
147        tigraycantwait  tigraygenocide           773     6211
174               eritrea          tigray           735    11428
4         stopwarontigray  tigraygenocide           681     6864
195    eritreaoutoftigray  tigraygenocide           622     3319
114               eritrea        ethiopia           615     7998
6875  allowaccesstotigray          tigray           568     8503
681           bidenactnow  tigraygenocide           548    14479
301              ethiopia  tigraygenocide           541     6632
691       bidentakeaction          tigray           506     4754
3317 

In [765]:
all_in_one(df_tigray,"mentions","tigray_mentions.html", weighting="Co_Occurence")


Creating Edgelist...


Most frequently co-occuring hashtags sorted by Co_Occurence:
              Entity_1       Entity_2  Co_Occurence  Retweet
19            refugees             un          2045    12206
20            refugees  unhumanrights          1498    13573
1105               hrw       refugees          1337     8642
1983           amnesty       refugees          1161    14280
3      antonioguterres       refugees          1046     5202
1107               hrw             un          1019    10910
83            refugees         unocha           926     4518
12169            potus       refugees           881     2892
257           joebiden       refugees           875     5634
4      antonioguterres             un           851     5322
82            refugees       ungeneva           818     2581
2065           amnesty            hrw           796     6522
9        filippograndi       refugees           785     9982
14070            potus             un           752     9966


In [766]:
all_in_one(df_rohingya,"hashtags","rohingya_hashtags.html", weighting="Co_Occurence")


Creating Edgelist...


Most frequently co-occuring hashtags sorted by Co_Occurence:
                    Entity_1                 Entity_2  Co_Occurence  Retweet
13                bangladesh                 rohingya           942    21172
7956               mar30coup  whatshappeninginmyanmar           574     9375
1440         milkteaalliance  whatshappeninginmyanmar           532    15335
896                    karen  whatshappeninginmyanmar           507    16567
15202     noflyzoneinmyanmar  whatshappeninginmyanmar           500    11277
52                   myanmar  whatshappeninginmyanmar           487    37565
11699               apr4coup  whatshappeninginmyanmar           464    15053
22                  refugees                 rohingya           424    10678
1492        internetshutdown  whatshappeninginmyanmar           380     5774
2                    myanmar                 rohingya           374    10212
108                  refugee                 rohingya           342 

In [767]:
all_in_one(df_rohingya,"mentions","rohingya_mentions.html", weighting="Co_Occurence")


Creating Edgelist...


Most frequently co-occuring hashtags sorted by Co_Occurence:
            Entity_1         Entity_2  Co_Occurence  Retweet
682         refugees               un           440      790
263     rapporteurun  schranerburgen1           412     1017
41          refugees           un_hrc           374      576
206         refugees    unhumanrights           360     1316
39      rapporteurun         refugees           359     4270
40      rapporteurun           un_hrc           313      632
318     rapporteurun               un           302      665
240       freya_cole     rapporteurun           295      572
343               un           un_hrc           281     1235
416         refugees  schranerburgen1           278      491
267  schranerburgen1           un_hrc           243      519
253          kenroth     rapporteurun           229      402
238       freya_cole          kenroth           221      410
332  schranerburgen1               un           210      442


## Create edgelist

In [704]:
df_edgelist_hash = create_edgelist(list(df_greece["hashtags"]), list(df_greece["retweet_count_sum"]))
df_edgelist_hash["Co_Occurence_Norm"] = df_edgelist_hash["Co_Occurence"]/(max(df_edgelist_hash["Co_Occurence"])/10)
df_edgelist_hash["Retweet_Norm"] = df_edgelist_hash["Retweet"]/(max(df_edgelist_hash["Retweet"])/10)
df_edgelist_hash

,Entity_1,Entity_2,Co_Occurence,Retweet,Co_Occurence_Norm,Retweet_Norm
0,aegeansea,amnestyinternational,1,1,0.002528,0.000224
1,aegeansea,europe,5,5,0.012642,0.001119
2,aegeansea,greece,22,66,0.055626,0.014772
3,aegeansea,greeklanguage,1,1,0.002528,0.000224
4,aegeansea,lesbos,5,7,0.012642,0.001567
...,...,...,...,...,...,...
108309,censor,immigration,1,3,0.002528,0.000671
108310,censor,refugees,1,3,0.002528,0.000671
108311,censor,refugeeswelcome,1,3,0.002528,0.000671
108312,eyesonidlib,immigration,1,3,0.002528,0.000671


In [692]:
#df_edgelist_hash = create_edgelist(list(df_greece["hashtags"])).sort_values(by="Weight",ascending=False).reset_index()
#df_edgelist_hash = df_edgelist_hash[df_edgelist_hash["Entity_1"] != df_edgelist_hash["Entity_2"]]

## Create graph

In [709]:
G_hashtag = nx.Graph()
for entity_1,entity_2,weight in zip(df_edgelist_hash["Entity_1"],
                                    df_edgelist_hash["Entity_2"],
                                    df_edgelist_hash["Co_Occurence_Norm"]):
    G_hashtag.add_edge(entity_1,entity_2, weight = weight, width=weight) #0.005

In [710]:
print(nx.info(G_hashtag))

Name: 
Type: Graph
Number of nodes: 11470
Number of edges: 108084
Average degree:  18.8464


## Provide node metadata

In [711]:
hashtag_count = get_entity_count(df_greece["hashtags"])

# Degree Centrality: 
degree_centrality = nx.degree_centrality(G_hashtag)

eigenvector_centrality = nx.eigenvector_centrality(G_hashtag, weight = "weight")

#size = {k:v*100 for (k,v) in degree_centrality.items()}
size = {k:v/max(degree_centrality.values())*10 for (k,v) in degree_centrality.items()}

In [773]:
?nx.degree_centrality

In [771]:
max(degree_centrality.values())

0.3601011422094341

In [712]:
nx.set_node_attributes(G_hashtag, hashtag_count, "hashtag_count" )
nx.set_node_attributes(G_hashtag, degree_centrality, "Degree Centrality")
nx.set_node_attributes(G_hashtag, eigenvector_centrality, "Eigenvector Centrality")
nx.set_node_attributes(G_hashtag, size, "size")

## Calculate network measures

In [713]:
# Density: Ratio of actual edges in the network to all possible edges
#density = nx.density(G_hashtag)
# Transitivity: Ration of all triangles over all possible triangles (triadic closure)
#trans = nx.transitivity(G_hashtag)
# Reciprocity:
#reci = nx.reciprocity(G_hashtag)

#print(f"Density: {density}\nTransitivity: {trans}\nReciprocity: {reci}")

In [714]:
# Diameter: Longest of all shortest paths in network
#nx.is_connected(G_hashtag)
# nx.diameter(G_hashtag)
# nx.average_shortest_path_length(G)

#components = nx.connected_components(G_hashtag)
#largest_component = max(components, key=len)
#subgraph = G_hashtag.subgraph(largest_component)
#diameter = nx.diameter(subgraph)
#print("Network diameter of largest component:", diameter)

## Create df with node attributes

In [718]:
df_node_attributes = pd.DataFrame.from_dict(dict(G_hashtag.nodes(data=True)),
                                            orient="index").reset_index().rename({"index":"hashtag"}, axis = 1)
df_node_attributes = df_node_attributes.sort_values(by="Degree Centrality", ascending=False).reset_index(drop=True)
df_node_attributes.head(15)

,hashtag,hashtag_count,Degree Centrality,Eigenvector Centrality,size
0,refugees,7346,0.360101,0.389795,10.000000
1,turkey,9362,0.305955,0.489616,8.496368
2,greece,10173,0.304037,0.453316,8.443099
3,syria,5982,0.262795,0.296607,7.297821
4,eu,4772,0.230099,0.259896,6.389831
5,europe,3388,0.222600,0.214648,6.181598
6,migrants,2796,0.185108,0.175692,5.140436
7,erdogan,2563,0.131746,0.177780,3.658596
8,refugee,1275,0.120063,0.057658,3.334140
9,idlib,2943,0.115878,0.157285,3.217918


## Create subgraph that only consists of top 50 nodes with highest degree centrality

In [656]:
cut_off = df_node_attributes.loc[50,"Degree Centrality"]
selected_nodes = [n for n,v in G_hashtag.nodes(data=True) if v["Degree Centrality"]>cut_off]

In [657]:
G_hashtag_sub = G_hashtag.subgraph(selected_nodes)

## Identify clusters

In [658]:
partition = community_louvain.best_partition(G_hashtag_sub, random_state=42)
print(f"Identified {len(set(partition.values()))} clusters.")

Identified 3 clusters.


In [659]:
colors_partition = {k:v for (k,v) in colors.items() if k < len(set(partition.values()))}

In [660]:
node_colors = {k:colors_partition[v] for (k,v) in partition.items()}

In [661]:
nx.set_node_attributes(G_hashtag_sub, partition, "Partiton")
nx.set_node_attributes(G_hashtag_sub, node_colors, "color")

## Draw network graph

In [662]:
G_hashtag_sub.nodes(data=True)

NodeDataView({'border': {'hashtag_count': 269, 'Degree Centrality': 0.041503182491934784, 'Eigenvector Centrality': 0.023235785355363452, 'size': 1.152542372881356, 'Partiton': 0, 'color': '#6495ED'}, 'refugeecrisis': {'hashtag_count': 546, 'Degree Centrality': 0.04429331240735897, 'Eigenvector Centrality': 0.03184868071036619, 'size': 1.2300242130750605, 'Partiton': 0, 'color': '#6495ED'}, 'migration': {'hashtag_count': 633, 'Degree Centrality': 0.07193303688202983, 'Eigenvector Centrality': 0.04369615384399717, 'size': 1.997578692493947, 'Partiton': 0, 'color': '#6495ED'}, 'russia': {'hashtag_count': 960, 'Degree Centrality': 0.07916993635016131, 'Eigenvector Centrality': 0.08673105296261695, 'size': 2.1985472154963683, 'Partiton': 1, 'color': '#9FE2BF'}, 'covid19': {'hashtag_count': 671, 'Degree Centrality': 0.05554102362891272, 'Eigenvector Centrality': 0.019062343801302905, 'size': 1.5423728813559323, 'Partiton': 0, 'color': '#6495ED'}, 'assad': {'hashtag_count': 576, 'Degree Cent

In [674]:
n = Network(height = "1500px", width ="100%", font_color ="#616a6b ")
#n.show_buttons()
n.from_nx(G_hashtag_sub)
n.barnes_hut()
n.show_buttons(filter_=["nodes","edges","physics"])
n.show("network_test.html")